In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import re
import json
import unicodedata
import numpy as np
import ast
from sklearn.impute import KNNImputer
warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath('../src'))

from data_preparation_utils import load_data
pd.set_option('display.max_columns', None)

In [3]:
from model_utils import prepare_data, convert_object_columns, target_encode_columns

# load/save data

In [5]:
# Load processed data
df = load_data(r"G:\My Drive\french_resturants\data\processed\french_resturants_only.csv")

In [37]:
#save data frame
df.to_csv(r"G:\My Drive\french_resturants\data\processed\french_resturants_ready_for_modeling.csv", index=False)

# prepare data for modeling

In [6]:
df.head()

,successful,region,province,city,claimed,awards,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,open_days_per_week,open_hours_per_week,working_shifts_per_week,default_language,accepts american express,accepts credit cards,accepts discover,accepts mastercard,accepts visa,afghani,african,after hours,albanian,algerian,american,apulian,arabic,argentinian,armenian,asian,assyrian,australian,austrian,azerbaijani,bahamian,bakeries,balti,bangladeshi,bar,barbecue,basque,beach,beer restaurants,beijing cuisine,beijing specialties,belgian,brazilian,breakfast,brew pub,british,brunch,buffet,burmese,byob,cafe,cajun creole,calabrian,cambodian,campania,canadian,cantonese,caribbean,cash only,catalan,caucasian,central american,central asian,central european,central italian,cheap eats,chilean,chinese,colombian,contemporary,costa rican,croatian,cuban,czech,danish,deli,delivery,delivery only,dessert,digital payments,dine with a local chef,diner,dining bars,dinner,dog friendly,drinks,drive thru,dutch,eastern european,ecuadorean,egyptian,emilian,ethiopian,european,family style,fast food,fine dining,free off street parking,free wifi,french,fruit parlours,fujian,full bar,fusion,gastropub,georgian,german,gift cards available,gluten free options,greek,grill,guangxi,guatemalan,halal,hawaiian,healthy,highchairs available,hokkaido cuisine,hong kong,hubei,hunan,hungarian,imperial chinese,indian,indonesian,international,irish,israeli,italian,jamaican,japanese,japanese fusion,japanese sweets parlour,jazz bar,kaiseki,kappo,korean,kosher,kyoto cuisine,kyushu cuisine,latin,latvian,lazio,lebanese,ligurian,live music,lombard,lunch,malaysian,medicinal foods,mediterranean,mexican,mid range,middle eastern,mongolian,moroccan,native american,neapolitan,nepalese,new zealand,nigerian,non smoking restaurants,northeastern chinese,northern italian,northwestern chinese,norwegian,outdoor seating,pakistani,parking available,persian,peruvian,philippine,pizza,playgrounds,polish,polynesian,portuguese,private dining,pub,puerto rican,quick bites,reservations,romagna,romana,romanian,russian,salvadoran,sardinian,scandinavian,scottish,seafood,seating,serves alcohol,shandong,shanghai,sicilian,singaporean,slovenian,soups,south american,southern italian,southwestern,spanish,speciality food market,sports bars,sri lankan,steakhouse,street food,street parking,sushi,swedish,swiss,szechuan,table service,taiwanese,takeout,television,thai,tibetan,tunisian,turkish,tuscan,ukrainian,uzbek,valet parking,validated parking,vegan options,vegetarian friendly,venezuelan,vietnamese,waterfront,welsh,wheelchair accessible,wine and beer,wine bar,xinjiang,yunnan,zhejiang,award_count,moved_location,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd,rank,total_restaurants_in_city
0,N,nouvelle aquitaine,haute vienne,saint jouvent,claimed,0 or unknown,"cheap eats, french",€,Unknown,"lunch, dinner",french,Missing,"reservations, seating, wheelchair accessible, ...",N,N,N,7,46.066667,14,english,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,0 or unknown,N,13:30,19:15,22:45,23:30,11:45,16:40,21:30,23:00,11:45,16:30,21:00,23:00,11:30,15:45,20:15,22:45,11:00,15:45,20:00,22:40,11:50,16:30,20:30,22:50,15:00,19:30,23:01,23:45,1.0,2.0

## trnsofrm columns type

In [9]:
# transform columns type
df = convert_object_columns(df, 
                            force_category=['region', 'province', 'city'], 
                            force_string=['top_tags', 'awards', 'meals', 'cuisines', 'special_diets', 'features'])

In [10]:
# Print summary of column data types
dtype_summary = df.dtypes.value_counts()
print("Summary of column data types:")
print(dtype_summary)

Summary of column data types:
boolean           232
string[python]      6
int64               2
category            1
category            1
category            1
category            1
category            1
category            1
float64             1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
category            1
Name: count, dtype: int6

In [11]:
# Select columns with boolean dtypes (including both native bool and pandas BooleanDtype)
bool_cols = df.select_dtypes(include=['bool', 'boolean']).columns

# Convert them to integer (0/1)
df[bool_cols] = df[bool_cols].astype(int)

In [14]:
df.head()

,successful,region,province,city,claimed,awards,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,open_days_per_week,open_hours_per_week,working_shifts_per_week,default_language,accepts american express,accepts credit cards,accepts discover,accepts mastercard,accepts visa,afghani,african,after hours,albanian,algerian,american,apulian,arabic,argentinian,armenian,asian,assyrian,australian,austrian,azerbaijani,bahamian,bakeries,balti,bangladeshi,bar,barbecue,basque,beach,beer restaurants,beijing cuisine,beijing specialties,belgian,brazilian,breakfast,brew pub,british,brunch,buffet,burmese,byob,cafe,cajun creole,calabrian,cambodian,campania,canadian,cantonese,caribbean,cash only,catalan,caucasian,central american,central asian,central european,central italian,cheap eats,chilean,chinese,colombian,contemporary,costa rican,croatian,cuban,czech,danish,deli,delivery,delivery only,dessert,digital payments,dine with a local chef,diner,dining bars,dinner,dog friendly,drinks,drive thru,dutch,eastern european,ecuadorean,egyptian,emilian,ethiopian,european,family style,fast food,fine dining,free off street parking,free wifi,french,fruit parlours,fujian,full bar,fusion,gastropub,georgian,german,gift cards available,gluten free options,greek,grill,guangxi,guatemalan,halal,hawaiian,healthy,highchairs available,hokkaido cuisine,hong kong,hubei,hunan,hungarian,imperial chinese,indian,indonesian,international,irish,israeli,italian,jamaican,japanese,japanese fusion,japanese sweets parlour,jazz bar,kaiseki,kappo,korean,kosher,kyoto cuisine,kyushu cuisine,latin,latvian,lazio,lebanese,ligurian,live music,lombard,lunch,malaysian,medicinal foods,mediterranean,mexican,mid range,middle eastern,mongolian,moroccan,native american,neapolitan,nepalese,new zealand,nigerian,non smoking restaurants,northeastern chinese,northern italian,northwestern chinese,norwegian,outdoor seating,pakistani,parking available,persian,peruvian,philippine,pizza,playgrounds,polish,polynesian,portuguese,private dining,pub,puerto rican,quick bites,reservations,romagna,romana,romanian,russian,salvadoran,sardinian,scandinavian,scottish,seafood,seating,serves alcohol,shandong,shanghai,sicilian,singaporean,slovenian,soups,south american,southern italian,southwestern,spanish,speciality food market,sports bars,sri lankan,steakhouse,street food,street parking,sushi,swedish,swiss,szechuan,table service,taiwanese,takeout,television,thai,tibetan,tunisian,turkish,tuscan,ukrainian,uzbek,valet parking,validated parking,vegan options,vegetarian friendly,venezuelan,vietnamese,waterfront,welsh,wheelchair accessible,wine and beer,wine bar,xinjiang,yunnan,zhejiang,award_count,moved_location,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd,rank,total_restaurants_in_city
0,0,nouvelle aquitaine,haute vienne,saint jouvent,claimed,0 or unknown,"cheap eats, french",€,Unknown,"lunch, dinner",french,Missing,"reservations, seating, wheelchair accessible, ...",0,0,0,7,46.066667,14,english,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0 or unknown,N,13:30,19:15,22:45,23:30,11:45,16:40,21:30,23:00,11:45,16:30,21:00,23:00,11:30,15:45,20:15,22:45,11:00,15:45,20:00,22:40,11:50,16:30,20:30,22:50,15:00,19:30,23:01,23:45,1.0,2.0

## target encoding for string columns

In [17]:
df['successful'].head()

0    0
1    0
2    0
3    0
4    0
Name: successful, dtype: int64

In [19]:
df = target_encode_columns(df, target_col="successful", columns=None)

## encoding rest of the data

In [22]:
for col in df.columns:
    # Check if the column is not numeric
    if not pd.api.types.is_numeric_dtype(df[col]):
        # Convert to category if not already
        if not pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].astype('category')
        # Replace column with its categorical codes
        df[col] = df[col].cat.codes

In [24]:
# Print summary of column data types
dtype_summary = df.dtypes.value_counts()
print("Summary of column data types:")
print(dtype_summary)

Summary of column data types:
int64      234
int8        35
float64      7
int16        4
Name: count, dtype: int64


# data split for modeling

In [27]:
X_train, X_test, y_train, y_test = prepare_data(df, 'successful')

In [29]:
X_train.head()

,region,province,city,claimed,awards,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,open_days_per_week,open_hours_per_week,working_shifts_per_week,default_language,accepts american express,accepts credit cards,accepts discover,accepts mastercard,accepts visa,afghani,african,after hours,albanian,algerian,american,apulian,arabic,argentinian,armenian,asian,assyrian,australian,austrian,azerbaijani,bahamian,bakeries,balti,bangladeshi,bar,barbecue,basque,beach,beer restaurants,beijing cuisine,beijing specialties,belgian,brazilian,breakfast,brew pub,british,brunch,buffet,burmese,byob,cafe,cajun creole,calabrian,cambodian,campania,canadian,cantonese,caribbean,cash only,catalan,caucasian,central american,central asian,central european,central italian,cheap eats,chilean,chinese,colombian,contemporary,costa rican,croatian,cuban,czech,danish,deli,delivery,delivery only,dessert,digital payments,dine with a local chef,diner,dining bars,dinner,dog friendly,drinks,drive thru,dutch,eastern european,ecuadorean,egyptian,emilian,ethiopian,european,family style,fast food,fine dining,free off street parking,free wifi,french,fruit parlours,fujian,full bar,fusion,gastropub,georgian,german,gift cards available,gluten free options,greek,grill,guangxi,guatemalan,halal,hawaiian,healthy,highchairs available,hokkaido cuisine,hong kong,hubei,hunan,hungarian,imperial chinese,indian,indonesian,international,irish,israeli,italian,jamaican,japanese,japanese fusion,japanese sweets parlour,jazz bar,kaiseki,kappo,korean,kosher,kyoto cuisine,kyushu cuisine,latin,latvian,lazio,lebanese,ligurian,live music,lombard,lunch,malaysian,medicinal foods,mediterranean,mexican,mid range,middle eastern,mongolian,moroccan,native american,neapolitan,nepalese,new zealand,nigerian,non smoking restaurants,northeastern chinese,northern italian,northwestern chinese,norwegian,outdoor seating,pakistani,parking available,persian,peruvian,philippine,pizza,playgrounds,polish,polynesian,portuguese,private dining,pub,puerto rican,quick bites,reservations,romagna,romana,romanian,russian,salvadoran,sardinian,scandinavian,scottish,seafood,seating,serves alcohol,shandong,shanghai,sicilian,singaporean,slovenian,soups,south american,southern italian,southwestern,spanish,speciality food market,sports bars,sri lankan,steakhouse,street food,street parking,sushi,swedish,swiss,szechuan,table service,taiwanese,takeout,television,thai,tibetan,tunisian,turkish,tuscan,ukrainian,uzbek,valet parking,validated parking,vegan options,vegetarian friendly,venezuelan,vietnamese,waterfront,welsh,wheelchair accessible,wine and beer,wine bar,xinjiang,yunnan,zhejiang,award_count,moved_location,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd,rank,total_restaurants_in_city
76179,13,31,8989,1,0.002820,0.001443,0,391,0.095365,0.003806,0.053901,0.001880,0,0,0,7,63.000000,7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,30,78,52,68,28,77,57,75,31,79,59,75,31,78,66,79,31,77,66,77,30,74,64,73,28,78,58,64,5895,118
78113,13,87,9225,1,0.681092,0.719178,2,391,0.262811,0.644444,0.581040,0.122514,1,0,1,6,19.000000,10,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [31]:
X_test.head()

,region,province,city,claimed,awards,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,open_days_per_week,open_hours_per_week,working_shifts_per_week,default_language,accepts american express,accepts credit cards,accepts discover,accepts mastercard,accepts visa,afghani,african,after hours,albanian,algerian,american,apulian,arabic,argentinian,armenian,asian,assyrian,australian,austrian,azerbaijani,bahamian,bakeries,balti,bangladeshi,bar,barbecue,basque,beach,beer restaurants,beijing cuisine,beijing specialties,belgian,brazilian,breakfast,brew pub,british,brunch,buffet,burmese,byob,cafe,cajun creole,calabrian,cambodian,campania,canadian,cantonese,caribbean,cash only,catalan,caucasian,central american,central asian,central european,central italian,cheap eats,chilean,chinese,colombian,contemporary,costa rican,croatian,cuban,czech,danish,deli,delivery,delivery only,dessert,digital payments,dine with a local chef,diner,dining bars,dinner,dog friendly,drinks,drive thru,dutch,eastern european,ecuadorean,egyptian,emilian,ethiopian,european,family style,fast food,fine dining,free off street parking,free wifi,french,fruit parlours,fujian,full bar,fusion,gastropub,georgian,german,gift cards available,gluten free options,greek,grill,guangxi,guatemalan,halal,hawaiian,healthy,highchairs available,hokkaido cuisine,hong kong,hubei,hunan,hungarian,imperial chinese,indian,indonesian,international,irish,israeli,italian,jamaican,japanese,japanese fusion,japanese sweets parlour,jazz bar,kaiseki,kappo,korean,kosher,kyoto cuisine,kyushu cuisine,latin,latvian,lazio,lebanese,ligurian,live music,lombard,lunch,malaysian,medicinal foods,mediterranean,mexican,mid range,middle eastern,mongolian,moroccan,native american,neapolitan,nepalese,new zealand,nigerian,non smoking restaurants,northeastern chinese,northern italian,northwestern chinese,norwegian,outdoor seating,pakistani,parking available,persian,peruvian,philippine,pizza,playgrounds,polish,polynesian,portuguese,private dining,pub,puerto rican,quick bites,reservations,romagna,romana,romanian,russian,salvadoran,sardinian,scandinavian,scottish,seafood,seating,serves alcohol,shandong,shanghai,sicilian,singaporean,slovenian,soups,south american,southern italian,southwestern,spanish,speciality food market,sports bars,sri lankan,steakhouse,street food,street parking,sushi,swedish,swiss,szechuan,table service,taiwanese,takeout,television,thai,tibetan,tunisian,turkish,tuscan,ukrainian,uzbek,valet parking,validated parking,vegan options,vegetarian friendly,venezuelan,vietnamese,waterfront,welsh,wheelchair accessible,wine and beer,wine bar,xinjiang,yunnan,zhejiang,award_count,moved_location,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd,rank,total_restaurants_in_city
95374,3,65,5207,2,0.002820,0.000000,1,391,0.050748,0.014944,0.053901,0.122514,0,0,0,7,108.000000,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,101,52,68,24,101,57,75,27,104,59,75,26,102,66,79,27,0,66,77,26,0,64,73,24,0,58,64,6313,116
14603,3,48,6236,1,0.002820,0.004396,0,2495,0.122928,0.007183,0.053901,0.122514,0,0,0,7,43.000000,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,

In [33]:
y_train.head()

76179    0
78113    1
58204    0
98299    0
5227     0
Name: successful, dtype: int64

In [35]:
y_test.head()

95374    0
14603    0
95722    0
21828    0
18746    0
Name: successful, dtype: int64

# model screening

In [40]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost

# Define your models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'XGBoost': xgboost.XGBRegressor(random_state=42)
}

# Prepare a list to store results
results = []

for name, model in models.items():
    # Fit the model on training data
    model.fit(X_train, y_train)
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    
    # For RMSLE, ensure no negative predictions (and target should be positive)
    y_pred_adj = np.maximum(y_pred, 0)
    y_test_adj = np.maximum(y_test, 0)
    rmsle = np.sqrt(mean_squared_error(np.log1p(y_test_adj), np.log1p(y_pred_adj)))
    
    results.append([name, mse, rmse, mae, rmsle])

# Convert the results list to a DataFrame for easy viewing
results_df = pd.DataFrame(results, columns=['Model', 'MSE', 'RMSE', 'MAE', 'RMSLE'])

# Sort results by RMSE (or another metric of your choice) and display
results_df = results_df.sort_values(by='RMSE', ascending=True)
print(results_df)

               Model       MSE      RMSE       MAE     RMSLE
6            XGBoost  0.030337  0.174176  0.065816  0.121505
4   GradientBoosting  0.031384  0.177157  0.067426  0.123994
2      Random Forest  0.031784  0.178279  0.063785  0.125272
0  Linear Regression  0.036736  0.191668  0.098328  0.134053
3           AdaBoost  0.046414  0.215438  0.110392  0.154224
1      Decision Tree  0.061197  0.247381  0.061197  0.171471
5                SVR  0.087899  0.296477  0.177475  0.207190


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score

# hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10, 15, 25],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 3, 5]
}

# Initialize the XGBRegressor with the appropriate objective and random state.
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Set up RandomizedSearchCV with a reduced number of iterations
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=10,  # fewer iterations
    scoring='neg_mean_squared_error',
    cv=5,       
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit RandomizedSearchCV on the training data
random_search.fit(X_train, y_train)

# Output the best hyperparameters.
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate on validation/test sets:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Test RMSE:", rmse)